In [31]:
!pip install gurobipy


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [32]:
import pandas as pd
from gurobipy import *
import numpy as np

## Load Data

In [33]:
# Load data files
child_care = pd.read_csv('child_care_regulated.csv')
population = pd.read_csv('population.csv')
income = pd.read_csv('avg_individual_income.csv')
employment = pd.read_csv('employment_rate.csv')
potential_locations = pd.read_csv('potential_locations.csv')

In [34]:
population.head()

,zipcode,Total,-5,5-9,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85+
0,6390,53,0,1,5,0,6,0,9,18,0,12,2,0,0,0,0,0,0,0
1,10001,27004,744,784,942,1035,2296,3806,3588,2524,1702,1903,1704,1225,1323,933,815,616,488,576
2,10002,76518,2142,3046,3198,2652,4528,6988,6278,5157,4962,4822,4410,6106,4548,4815,4748,2531,2793,2794
3,10003,53877,1440,1034,953,7013,6344,7100,6427,3221,2907,1988,2698,2350,2274,2793,1854,1646,779,1056
4,10004,4579,433,182,161,108,109,601,724,490,241,313,549,279,199,173,2,15,0,0


## Pre-process

In [35]:
# Rename zip_code columns for consistency
population.rename(columns={'zipcode': 'zip_code'}, inplace=True)
income.rename(columns={'ZIP code': 'zip_code'}, inplace=True)
employment.rename(columns={'zipcode': 'zip_code'}, inplace=True)

# under5 = infant + toddler + preschool
child_care['under5_capacity'] = (
    child_care['infant_capacity'] +
    child_care['toddler_capacity'] +
    child_care['preschool_capacity']
)

# Aggregate child care data to zip_code level
child_care_grouped = (
    child_care.groupby('zip_code')[['total_capacity', 'under5_capacity']]
    .sum()
    .reset_index()
)

# Merge everything to create area_df (one row per zip_code)
area_df = population.merge(child_care_grouped, on='zip_code', how='left')
area_df = area_df.merge(income, on='zip_code', how='left')
area_df = area_df.merge(employment, on='zip_code', how='left')

# Fill NaN with 0
area_df[['total_capacity', 'under5_capacity']] = area_df[['total_capacity', 'under5_capacity']].fillna(0)

# Compute total child population 
area_df['under5_children'] = area_df['-5']
area_df['total_children'] = area_df['-5'] + area_df['5-9'] + 0.6 * area_df['10-14']

# Classify high demand areas
area_df['high_demand'] = (area_df['employment rate'] >= 60) | (area_df['average income'] <= 60000)

# Compute coverage ratios
area_df['coverage_ratio'] = area_df['total_capacity'] / area_df['total_children']
area_df['under5_coverage_ratio'] = area_df['under5_capacity'] / area_df['under5_children']

# Identify child care deserts
area_df['is_desert'] = (
    (area_df['high_demand'] & (area_df['coverage_ratio'] <= 0.5)) |
    (~area_df['high_demand'] & (area_df['coverage_ratio'] <= 1/3))
)

# Identify under-5 deserts
area_df['is_under5_desert'] = area_df['under5_coverage_ratio'] < (2/3)

# Drop irrelevant columns
columns_to_keep = [
    'zip_code', 'total_children', 'under5_children', 'total_capacity', 'under5_capacity',
    'average income', 'employment rate', 'high_demand', 'is_desert', 'is_under5_desert'
]
area_df = area_df[columns_to_keep]

In [36]:
area_df

,zip_code,total_children,under5_children,total_capacity,under5_capacity,average income,employment rate,high_demand,is_desert,is_under5_desert
0,6390,4.0,0,0.0,0.0,NaN,NaN,False,True,False
1,10001,2093.2,744,609.0,0.0,102878.033603,0.595097,False,True,True
2,10002,7106.8,2142,4729.0,18.0,59604.041165,0.520662,True,False,True
3,10003,3045.8,1440,1995.0,0.0,114273.049645,0.497244,False,False,True
4,10004,711.6,433,263.0,0.0,132004.310345,0.506661,False,False,True
...,...,...,...,...,...,...,...,...,...,...
1641,14774,68.8,4,62.0,0.0,NaN,NaN,False,False,True
1642,14785,0.0,0,0.0,0.0,NaN,NaN,False,False,False
1643,14788,0.6,0,0.0,0.0,NaN,NaN,False,True,False
1644,14805,64.4,31,8.0,0.0,59375.000000,0.679739,True,True,True


In [37]:
from gurobipy import Model, GRB, quicksum

A = area_df['zip_code'].tolist()

Ah = area_df[area_df['high_demand']]['zip_code'].tolist()
An = area_df[~area_df['high_demand']]['zip_code'].tolist()

pta = area_df.set_index('zip_code')['total_children'].to_dict()
pua = area_df.set_index('zip_code')['under5_children'].to_dict()
existing_total = area_df.set_index('zip_code')['total_capacity'].to_dict()
existing_under5 = area_df.set_index('zip_code')['under5_capacity'].to_dict()

build_cost = {1: 65000, 2: 95000, 3: 115000}
capacity_all = {1: 100, 2: 200, 3: 400}
capacity_u5 = {1: 50, 2: 100, 3: 200}
extra_u5_cost = 100



In [38]:
def solve_model(expansion_slope):
    model = Model(f"ChildCare_Slope_{expansion_slope}")

    y = model.addVars(A, [1, 2, 3], vtype=GRB.INTEGER, name="y")
    mt = model.addVars(A, vtype=GRB.CONTINUOUS, name="mt")
    mu = model.addVars(A, vtype=GRB.CONTINUOUS, name="mu")

    model.setObjective(
        quicksum(build_cost[i] * y[a, i] for a in A for i in [1, 2, 3]) +
        quicksum((20000 + expansion_slope * existing_total[a]) * (mt[a] / max(existing_total[a], 1)) for a in A) +
        quicksum(extra_u5_cost * mu[a] for a in A),
        GRB.MINIMIZE
    )

    for a in Ah:
        model.addConstr(existing_total[a] + mt[a] + quicksum(capacity_all[i] * y[a, i] for i in [1, 2, 3]) >= 0.5 * pta[a])
    for a in An:
        model.addConstr(existing_total[a] + mt[a] + quicksum(capacity_all[i] * y[a, i] for i in [1, 2, 3]) >= (1/3) * pta[a])
    for a in A:
        model.addConstr(existing_under5[a] + mu[a] + quicksum(capacity_u5[i] * y[a, i] for i in [1, 2, 3]) >= (2/3) * pua[a])
        model.addConstr(mu[a] <= mt[a])
        model.addConstr(mt[a] <= 0.2 * existing_total[a])

    model.optimize()

    if model.status == GRB.OPTIMAL:
        print(f"\n✅ Slope = {expansion_slope}: Optimal Cost = ${model.objVal:,.2f}\n")
        for a in A:
            s = int(y[a, 1].X) if y[a, 1].X else 0
            m = int(y[a, 2].X) if y[a, 2].X else 0
            l = int(y[a, 3].X) if y[a, 3].X else 0
            base = existing_total[a]
            ratio = mt[a].X / base if base > 0 else 0
            print(f"{a} → small: {s}, medium: {m}, large: {l}, expand: {ratio * 100:.2f}%, under5: {mu[a].X:.0f}")
        return model.objVal
    else:
        print("No optimal solution found.")
        return None

In [ ]:
for slope in [200, 400, 1000]:
    cost = solve_model(slope)


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.3.0 24D81)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6584 rows, 8230 columns and 18106 nonzeros
Model fingerprint: 0x6b04f12d
Variable types: 3292 continuous, 4938 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [1e+02, 1e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 1e+04]
Found heuristic solution: objective 4.506850e+08
Presolve removed 6581 rows and 8225 columns
Presolve time: 0.10s
Presolved: 3 rows, 5 columns, 10 nonzeros
Found heuristic solution: objective 3.720945e+08
Variable types: 0 continuous, 5 integer (0 binary)

Root relaxation: objective 3.719852e+08, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     

## Final Choice(with minimum cost): slope = 200

In [40]:
solve_model(200)

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.3.0 24D81)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6584 rows, 8230 columns and 18106 nonzeros
Model fingerprint: 0x6b04f12d
Variable types: 3292 continuous, 4938 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [1e+02, 1e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 1e+04]
Found heuristic solution: objective 4.506850e+08
Presolve removed 6581 rows and 8225 columns
Presolve time: 0.11s
Presolved: 3 rows, 5 columns, 10 nonzeros
Found heuristic solution: objective 3.720945e+08
Variable types: 0 continuous, 5 integer (0 binary)

Root relaxation: objective 3.719852e+08, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     

372015630.1573953